# Areal interpolation: Tracts to Voting Precincts

This notebook demonstrates the interpolation of an intensive variable (Pct Youth) measured for the census tracts tracts in Riverside and San Bernardino counties in California to the voting precincts in the respective counties.

In [ ]:
import tobler
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
import geopandas

In [ ]:
tracts = geopandas.read_file("https://ndownloader.figshare.com/files/20460645") 

In [ ]:
tracts.shape

There are 822 tracts in the two counties.

In [ ]:
tracts.head()

In [ ]:
tracts.plot(facecolor='none', edgecolor='g')

## Precincts

In [ ]:
precincts = geopandas.read_file("https://ndownloader.figshare.com/files/20460549") 


In [ ]:
precincts.shape

For the 3780 precincts in the two counties, we wish to obtain estimates of the percentage of the  population that is youth.

In [ ]:
precincts.plot(facecolor='none', edgecolor='r')

## Interpolation

In [ ]:
estimates = tobler.area_weighted.area_interpolate(tracts, precincts, intensive_variables=['pct Youth'])

Notice the warning about different crs. 

In [ ]:
estimates

As a result of the different crs, tobler will not carry out the interpolation. We need to fix the crs issue first by setting the tract geometries to use the precincts crs

In [ ]:
tracts = tracts.to_crs(precincts.crs) 

In [ ]:
estimates = tobler.area_weighted.area_interpolate(tracts, precincts, intensive_variables=['pct Youth'])

In [ ]:
estimates

In [ ]:
f, ax = plt.subplots(1, figsize=(8, 8))
ax = tracts.plot(column='pct Youth', ax=ax, legend=True, alpha=0.5, scheme='Quantiles', k=10)
plt.show()

In [ ]:
precincts['pct Youth'] = estimates['pct Youth']
f, ax = plt.subplots(1, figsize=(8, 8))
ax = precincts.plot(column='pct Youth', ax=ax, legend=True, alpha=0.5, scheme='Quantiles', k=10)
plt.show()